In [ ]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from modules._import_helper import safe_import_from

# Import hybrid models
(MeasurementNetwork, HybridEKF, HybridEKFConfig, HybridParticleFilter,
 create_nonlinear_measurement_system, generate_training_data) = safe_import_from(
    '04_time_series_state_space.src.hybrid_models',
    'MeasurementNetwork', 'HybridEKF', 'HybridEKFConfig', 'HybridParticleFilter',
    'create_nonlinear_measurement_system', 'generate_training_data'
)

# Import classical filters
ExtendedKalmanFilter = safe_import_from(
    '04_time_series_state_space.src.ekf', 'ExtendedKalmanFilter'
)
ParticleFilter, gaussian_likelihood, create_process_noise_wrapper = safe_import_from(
    '04_time_series_state_space.src.particle_filter',
    'ParticleFilter', 'gaussian_likelihood', 'create_process_noise_wrapper'
)

set_seed = safe_import_from('00_repo_standards.src.mlphys_core', 'set_seed')

set_seed(42)
torch.manual_seed(42)
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Problem Setup: Complex Measurement Function

We simulate a system where:
- **Dynamics**: Linear (constant velocity) - known physics
- **Measurement**: Nonlinear - unknown, must be learned

$$h(x) = \sin(x_0) + 0.5 \cdot x_0 \cdot x_1$$

This is challenging because:
1. Linear approximation is poor
2. Standard EKF requires known Jacobian
3. Product term couples position and velocity

In [ ]:
# System parameters
dt = 0.1
n_steps = 100
process_noise = 0.02
obs_noise = 0.1

# Dynamics: constant velocity (linear)
F = np.array([[1, dt], [0, 1]])
Q = np.eye(2) * process_noise**2
R = np.array([[obs_noise**2]])

def f_dynamics(x, u):
    return F @ x

def F_jacobian(x):
    return F

# Measurement: complex nonlinear function
h_true, h_linear, h_jac_true = create_nonlinear_measurement_system("moderate")

print("System Setup:")
print(f"  Dynamics: Linear (constant velocity), dt={dt}")
print(f"  Measurement: h(x) = sin(x[0]) + 0.5*x[0]*x[1]")
print(f"  Process noise: {process_noise}")
print(f"  Observation noise: {obs_noise}")

# Visualize measurement function
x0_range = np.linspace(-2, 2, 50)
x1_vals = [0, 0.5, 1.0]

plt.figure(figsize=(10, 4))
for x1 in x1_vals:
    z_vals = [h_true(np.array([x0, x1]))[0] for x0 in x0_range]
    plt.plot(x0_range, z_vals, label=f'x₁={x1}')

plt.plot(x0_range, x0_range, 'k--', label='Linear (h=x₀)')
plt.xlabel('x₀ (position)')
plt.ylabel('Measurement h(x)')
plt.title('True Measurement Function (highly nonlinear)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 2. Train the Measurement Network

Before filtering, we train a neural network to approximate $h(x)$.

**Important**: Training data must cover the expected state space!

In [ ]:
# Generate training data
rng = np.random.default_rng(42)
n_train = 500

X_train, Z_train = generate_training_data(
    f_dynamics=None,
    h_true=h_true,
    n_samples=n_train,
    noise_std=obs_noise,
    rng=rng,
    x_range=(-3, 3),  # Cover expected state range
)

print(f"Training data: {X_train.shape[0]} samples")
print(f"State range: [{X_train.min():.2f}, {X_train.max():.2f}]")

In [ ]:
# Setup and train hybrid EKF
config = HybridEKFConfig(
    n_states=2,
    n_obs=1,
    hidden_dims=[64, 32],  # Larger network for complex function
    learning_rate=0.005,
    n_epochs=200,
    batch_size=32,
)

hybrid_ekf = HybridEKF(f_dynamics, F_jacobian, Q, R, config)

print("Training measurement network...")
losses = hybrid_ekf.train_measurement_model(X_train, Z_train, verbose=True)

plt.figure(figsize=(10, 4))
plt.semilogy(losses)
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.title('Measurement Network Training')
plt.grid(True, alpha=0.3)
plt.show()

print(f"\nFinal loss: {losses[-1]:.6f}")

In [ ]:
# Evaluate learned measurement function
test_states = np.array([
    [0, 0], [0.5, 0.5], [-0.5, 0.5], [1, 1], [-1, -1]
])

print("Learned vs True Measurement:")
print("-" * 50)
print(f"{'State':^20} {'True':^12} {'Learned':^12} {'Error':^10}")
print("-" * 50)

for x in test_states:
    z_true = h_true(x)[0]
    z_learned = hybrid_ekf.h_network.predict_numpy(x)[0]
    error = abs(z_true - z_learned)
    print(f"{str(x):^20} {z_true:^12.4f} {z_learned:^12.4f} {error:^10.4f}")

## 3. Comparison: Hybrid EKF vs Classical Methods

We compare:
1. **Hybrid EKF**: Learned measurement + physics dynamics
2. **Classical EKF (true)**: Uses true Jacobian (oracle baseline)
3. **Classical EKF (linear)**: Assumes linear measurement (wrong model)
4. **Raw observations**: No filtering

In [ ]:
# Generate ground truth trajectory
set_seed(123)  # Different seed from training
rng_test = np.random.default_rng(123)

# True trajectory
true_states = []
observations = []
x_true = np.array([0.0, 1.0])  # Start at origin with velocity 1

for _ in range(n_steps):
    # Dynamics with noise
    w = rng_test.multivariate_normal(np.zeros(2), Q)
    x_true = F @ x_true + w
    true_states.append(x_true.copy())
    
    # Nonlinear measurement with noise
    v = rng_test.normal(0, obs_noise)
    z = h_true(x_true) + v
    observations.append(z[0])

true_states = np.array(true_states)
observations = np.array(observations)

print(f"Generated {n_steps} timesteps")
print(f"Position range: [{true_states[:, 0].min():.2f}, {true_states[:, 0].max():.2f}]")
print(f"Velocity range: [{true_states[:, 1].min():.2f}, {true_states[:, 1].max():.2f}]")

In [ ]:
# Run Hybrid EKF
hybrid_ekf.initialize(x0=np.array([0, 0]), P0=np.eye(2))
hybrid_estimates = []

for z in observations:
    hybrid_ekf.predict()
    hybrid_ekf.update(np.array([z]))
    x, _ = hybrid_ekf.get_state()
    hybrid_estimates.append(x)

hybrid_estimates = np.array(hybrid_estimates)

# Run Classical EKF with TRUE Jacobian (oracle)
def h_true_wrapper(x):
    return h_true(x)

ekf_true = ExtendedKalmanFilter(f_dynamics, h_true_wrapper, F_jacobian, h_jac_true, Q, R)
ekf_true.initialize(x0=np.array([0, 0]), P0=np.eye(2))
ekf_true_estimates = []

for z in observations:
    ekf_true.predict()
    ekf_true.update(np.array([z]))
    x, _ = ekf_true.get_state()
    ekf_true_estimates.append(x)

ekf_true_estimates = np.array(ekf_true_estimates)

# Run Classical EKF with LINEAR measurement (wrong model)
def h_linear_wrapper(x):
    return h_linear(x)

def h_linear_jac(x):
    return np.array([[1, 0]])  # Assumes h = x[0]

ekf_linear = ExtendedKalmanFilter(f_dynamics, h_linear_wrapper, F_jacobian, h_linear_jac, Q, R)
ekf_linear.initialize(x0=np.array([0, 0]), P0=np.eye(2))
ekf_linear_estimates = []

for z in observations:
    ekf_linear.predict()
    ekf_linear.update(np.array([z]))
    x, _ = ekf_linear.get_state()
    ekf_linear_estimates.append(x)

ekf_linear_estimates = np.array(ekf_linear_estimates)

print("Filtering complete!")

In [ ]:
# Compute metrics
def compute_rmse(true, estimated):
    return np.sqrt(np.mean((true - estimated)**2))

rmse_hybrid = compute_rmse(true_states, hybrid_estimates)
rmse_ekf_true = compute_rmse(true_states, ekf_true_estimates)
rmse_ekf_linear = compute_rmse(true_states, ekf_linear_estimates)

# Position RMSE only
rmse_hybrid_pos = compute_rmse(true_states[:, 0], hybrid_estimates[:, 0])
rmse_ekf_true_pos = compute_rmse(true_states[:, 0], ekf_true_estimates[:, 0])
rmse_ekf_linear_pos = compute_rmse(true_states[:, 0], ekf_linear_estimates[:, 0])

print("\n" + "="*60)
print("RESULTS: RMSE Comparison")
print("="*60)
print(f"{'Method':<30} {'Total RMSE':>12} {'Position RMSE':>15}")
print("-"*60)
print(f"{'Hybrid EKF (learned h)':<30} {rmse_hybrid:>12.4f} {rmse_hybrid_pos:>15.4f}")
print(f"{'EKF with true Jacobian':<30} {rmse_ekf_true:>12.4f} {rmse_ekf_true_pos:>15.4f}")
print(f"{'EKF with linear h (wrong!)':<30} {rmse_ekf_linear:>12.4f} {rmse_ekf_linear_pos:>15.4f}")
print("="*60)

# Improvement ratio
improvement = (rmse_ekf_linear - rmse_hybrid) / rmse_ekf_linear * 100
print(f"\nHybrid improves over wrong-model EKF by {improvement:.1f}%")

In [ ]:
# Visualization
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
times = np.arange(n_steps) * dt

# Position tracking
ax = axes[0, 0]
ax.plot(times, true_states[:, 0], 'k-', label='True', linewidth=2)
ax.plot(times, hybrid_estimates[:, 0], 'r--', label='Hybrid EKF', linewidth=2)
ax.plot(times, ekf_true_estimates[:, 0], 'g:', label='EKF (true h)', linewidth=2)
ax.plot(times, ekf_linear_estimates[:, 0], 'b-.', label='EKF (linear h)', linewidth=1.5, alpha=0.7)
ax.set_ylabel('Position')
ax.legend()
ax.set_title('Position Tracking')
ax.grid(True, alpha=0.3)

# Velocity tracking
ax = axes[0, 1]
ax.plot(times, true_states[:, 1], 'k-', label='True', linewidth=2)
ax.plot(times, hybrid_estimates[:, 1], 'r--', label='Hybrid EKF', linewidth=2)
ax.plot(times, ekf_true_estimates[:, 1], 'g:', label='EKF (true h)', linewidth=2)
ax.plot(times, ekf_linear_estimates[:, 1], 'b-.', label='EKF (linear h)', linewidth=1.5, alpha=0.7)
ax.set_ylabel('Velocity')
ax.legend()
ax.set_title('Velocity Tracking')
ax.grid(True, alpha=0.3)

# Position error
ax = axes[1, 0]
ax.plot(times, np.abs(true_states[:, 0] - hybrid_estimates[:, 0]), 'r-', label='Hybrid EKF', linewidth=2)
ax.plot(times, np.abs(true_states[:, 0] - ekf_true_estimates[:, 0]), 'g-', label='EKF (true h)', linewidth=2)
ax.plot(times, np.abs(true_states[:, 0] - ekf_linear_estimates[:, 0]), 'b-', label='EKF (linear h)', linewidth=1.5, alpha=0.7)
ax.set_xlabel('Time (s)')
ax.set_ylabel('Absolute Error')
ax.legend()
ax.set_title('Position Estimation Error')
ax.grid(True, alpha=0.3)

# RMSE bar chart
ax = axes[1, 1]
methods = ['Hybrid\nEKF', 'EKF\n(true h)', 'EKF\n(linear h)']
rmses = [rmse_hybrid, rmse_ekf_true, rmse_ekf_linear]
colors = ['red', 'green', 'blue']
bars = ax.bar(methods, rmses, color=colors, alpha=0.7)
ax.set_ylabel('RMSE')
ax.set_title('Overall RMSE Comparison')
for bar, rmse in zip(bars, rmses):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
            f'{rmse:.3f}', ha='center', va='bottom', fontsize=11)

plt.tight_layout()
plt.show()

## 4. Failure Mode Analysis

Let's examine when hybrid models **fail**.

### Failure Mode 1: Out-of-Distribution States

The NN was trained on states in $[-3, 3]$. What happens outside this range?

In [ ]:
# Test extrapolation
ood_states = np.array([
    [5, 0],    # Outside training range
    [-5, 0],
    [0, 5],
    [10, 10],  # Very far out
])

print("Out-of-Distribution Performance:")
print("-" * 60)
print(f"{'State':^20} {'True':^12} {'Learned':^12} {'Error':^10}")
print("-" * 60)

for x in ood_states:
    z_true = h_true(x)[0]
    z_learned = hybrid_ekf.h_network.predict_numpy(x)[0]
    error = abs(z_true - z_learned)
    status = "⚠️" if error > 0.5 else "✓"
    print(f"{str(x):^20} {z_true:^12.4f} {z_learned:^12.4f} {error:^10.4f} {status}")

print("\n⚠️ Warning: NN extrapolates poorly outside training data!")

### Failure Mode 2: Insufficient Training Data

In [ ]:
# Train with very little data
small_data_sizes = [20, 50, 100, 200, 500]
rmses_vs_data = []

for n_train in small_data_sizes:
    torch.manual_seed(42)
    rng_small = np.random.default_rng(42)
    
    X_small, Z_small = generate_training_data(
        None, h_true, n_samples=n_train, noise_std=obs_noise, rng=rng_small
    )
    
    config_small = HybridEKFConfig(
        n_states=2, n_obs=1, hidden_dims=[32, 16],
        learning_rate=0.01, n_epochs=100
    )
    hybrid_small = HybridEKF(f_dynamics, F_jacobian, Q, R, config_small)
    hybrid_small.train_measurement_model(X_small, Z_small)
    
    # Test on same trajectory
    hybrid_small.initialize(x0=np.array([0, 0]), P0=np.eye(2))
    estimates = []
    for z in observations:
        hybrid_small.predict()
        hybrid_small.update(np.array([z]))
        x, _ = hybrid_small.get_state()
        estimates.append(x)
    
    rmse = compute_rmse(true_states, np.array(estimates))
    rmses_vs_data.append(rmse)

plt.figure(figsize=(10, 5))
plt.semilogx(small_data_sizes, rmses_vs_data, 'o-', markersize=10, linewidth=2)
plt.axhline(rmse_ekf_true, color='g', linestyle='--', label='EKF (true h)')
plt.axhline(rmse_ekf_linear, color='b', linestyle='--', label='EKF (linear h)')
plt.xlabel('Training Data Size')
plt.ylabel('RMSE')
plt.title('Hybrid EKF Performance vs Training Data Size')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print("\n📊 Data size effect:")
for n, rmse in zip(small_data_sizes, rmses_vs_data):
    print(f"  n={n:4d}: RMSE = {rmse:.4f}")

### Failure Mode 3: Simple Systems (NN Not Needed)

What if the measurement IS actually linear?

In [ ]:
# Generate LINEAR measurement data
torch.manual_seed(42)
rng_linear = np.random.default_rng(42)

def h_actually_linear(x):
    return np.array([x[0]])

X_lin, Z_lin = generate_training_data(
    None, h_actually_linear, n_samples=500, noise_std=obs_noise, rng=rng_linear
)

# Train hybrid on linear data
config_lin = HybridEKFConfig(
    n_states=2, n_obs=1, hidden_dims=[64, 32],
    learning_rate=0.005, n_epochs=200
)
hybrid_lin = HybridEKF(f_dynamics, F_jacobian, Q, R, config_lin)
hybrid_lin.train_measurement_model(X_lin, Z_lin)

# Generate test trajectory with linear measurements
rng_test_lin = np.random.default_rng(456)
obs_linear = []
for state in true_states:
    z = h_actually_linear(state) + rng_test_lin.normal(0, obs_noise)
    obs_linear.append(z[0])

# Run hybrid EKF
hybrid_lin.initialize(x0=np.array([0, 0]), P0=np.eye(2))
hybrid_lin_est = []
for z in obs_linear:
    hybrid_lin.predict()
    hybrid_lin.update(np.array([z]))
    x, _ = hybrid_lin.get_state()
    hybrid_lin_est.append(x)

# Run standard EKF (linear h)
ekf_lin = ExtendedKalmanFilter(f_dynamics, h_actually_linear, F_jacobian, h_linear_jac, Q, R)
ekf_lin.initialize(x0=np.array([0, 0]), P0=np.eye(2))
ekf_lin_est = []
for z in obs_linear:
    ekf_lin.predict()
    ekf_lin.update(np.array([z]))
    x, _ = ekf_lin.get_state()
    ekf_lin_est.append(x)

rmse_hybrid_lin = compute_rmse(true_states, np.array(hybrid_lin_est))
rmse_ekf_lin = compute_rmse(true_states, np.array(ekf_lin_est))

print("\n" + "="*50)
print("SIMPLE (LINEAR) SYSTEM - Hybrid vs Classical")
print("="*50)
print(f"Hybrid EKF RMSE:   {rmse_hybrid_lin:.4f}")
print(f"Standard EKF RMSE: {rmse_ekf_lin:.4f}")
print(f"\nRatio (Hybrid/Standard): {rmse_hybrid_lin/rmse_ekf_lin:.2f}x")
print("\n⚠️ For simple systems, classical methods are optimal.")
print("   NN adds unnecessary complexity and slight error.")

## 5. Summary: When to Use Hybrid Models

### ✅ Use Hybrid Models When:
1. Measurement/dynamics function is **unknown or complex**
2. Sufficient training data is **available**
3. States will stay **within training distribution**
4. Classical linearization (EKF) **performs poorly**

### ❌ Don't Use When:
1. System is **well-modeled** by physics (exact equations known)
2. **Limited training data** available
3. States may go **out of distribution**
4. Classical methods work **well enough**

### 📊 Key Metrics to Track:
- **Training loss convergence** (should decrease)
- **RMSE comparison** vs. classical baselines
- **Out-of-distribution performance** (test on edge cases)
- **Data efficiency** (how much data needed?)

In [ ]:
# Final summary table
print("\n" + "="*70)
print("FINAL SUMMARY: Hybrid vs Classical State Estimation")
print("="*70)
print(f"\n{'Scenario':<35} {'Best Method':<25}")
print("-"*70)
print(f"{'Complex nonlinear measurement':<35} {'Hybrid EKF':<25}")
print(f"{'Unknown sensor model':<35} {'Hybrid EKF (learn h)':<25}")
print(f"{'Linear/simple system':<35} {'Classical Kalman Filter':<25}")
print(f"{'Limited training data':<35} {'Classical EKF':<25}")
print(f"{'Wide state range (extrapolation)':<35} {'Classical EKF + physics':<25}")
print(f"{'Non-Gaussian noise':<35} {'Hybrid Particle Filter':<25}")
print("="*70)